In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
import random
import time
from numpy import array
import keras
import sys

Using TensorFlow backend.


In [0]:
tf.reset_default_graph()
x1 = tf.placeholder(tf.float32)
y1 = tf.placeholder(tf.float32)
z1 = tf.placeholder(tf.float32)
x2 = tf.placeholder(tf.float32)
y2 = tf.placeholder(tf.float32)
z2 = tf.placeholder(tf.float32)
cube = tf.placeholder(tf.float32)
lr = tf.placeholder(tf.float32)
Y_ = tf.placeholder(tf.int32)

x1 = tf.reshape(x1, shape=[1,29,29,1])
y1 = tf.reshape(y1, shape=[1,29,29,1])
z1 = tf.reshape(z1, shape=[1,29,29,1])
x2 = tf.reshape(x2, shape=[1,87,87,1])
y2 = tf.reshape(y2, shape=[1,87,87,1])
z2 = tf.reshape(z2, shape=[1,87,87,1])
cube = tf.reshape(cube, shape=[1,13,13,13,1])

In [0]:
def conv3d(x, n_kernels):
  return tf.layers.conv3d(x,n_kernels,(5,5,5),padding="valid",activation=tf.nn.elu)

def conv2d(x, n_kernels):
  return tf.layers.conv2d(x,n_kernels,(5,5),padding="valid",activation=tf.nn.elu)

def conv2d_shared(x, n_kernels,nam):
  return tf.layers.conv2d(x,n_kernels,(5,5),padding="valid",name=nam)

def maxpool(x):
  return tf.layers.max_pooling2d(x,(2,2),padding="valid")

def meanpool(x):
  return tf.layers.average_pooling2d(x,(3,3),strides=3)

def batchnorm(x):
  return tf.contrib.layers.batch_norm(x, decay= 0.99, is_training=True,center= True, scale=True, reuse= False)
    
def relu(x):
  return tf.nn.relu(x)

def residual(x,y):
  n_x = x.shape[-1]
  n_y = y.shape[-1]
  spatial_rank = 3

  if n_x > n_y:  # pad the channel dim
      pad_1 = np.int((n_x - n_y) // 2)
      pad_2 = np.int(n_x - n_y - pad_1)
      padding_dims = np.vstack(([[0, 0]], [[0, 0]] * spatial_rank, [[pad_1, pad_2]]))
      y = tf.pad(tensor=y, paddings=padding_dims.tolist(), mode='CONSTANT')
  elif n_x < n_y:  # make a projection
      y = conv3d(y,1)

  # element-wise sum of both paths
  output_tensor = x + y
  

  return output_tensor

def dilatedCon(x,n_kernels,dilation):
  return tf.layers.conv3d(x, n_kernels,(3,3,3), [1, 1, 1], padding="same", dilation_rate=[dilation, dilation, dilation])

def finalconv(x):
  return tf.layers.conv3d(x,1,(1,1,1),padding="same")

def softmax(x):
  return tf.nn.softmax(x)

In [0]:
with tf.device('/gpu:0'):
  with tf.variable_scope("conv1",reuse=tf.AUTO_REUSE):
    x1_1 = conv2d_shared(x1,20,"layer1")
    y1_1 = conv2d_shared(y1,20,"layer1")
    z1_1 = conv2d_shared(z1,20,"layer1")
  with tf.variable_scope("conv2",reuse=tf.AUTO_REUSE):
    x2_1 = conv2d_shared(meanpool(x2),20,"layer2")
    y2_1 = conv2d_shared(meanpool(y2),20,"layer2")
    z2_1 = conv2d_shared(meanpool(z2),20,"layer2")

  cube_1 = conv3d(cube,20)

  x1_2 = conv2d(x1_1,50)
  y1_2 = conv2d(y1_1,50)
  z1_2 = conv2d(z1_1,50)
  x2_2 = conv2d(x2_1,50)
  y2_2 = conv2d(y2_1,50)
  z2_2 = conv2d(z2_1,50)
  cube_2 = conv3d(cube_1,50)

  x1_2 = tf.layers.flatten(x1_2)
  y1_2 = tf.layers.flatten(y1_2)
  z1_2 = tf.layers.flatten(z1_2)
  x2_2 = tf.layers.flatten(x2_2)
  y2_2 = tf.layers.flatten(y2_2)
  z2_2 = tf.layers.flatten(z2_2)
  cube_2 = tf.layers.flatten(cube_2)

  conv = tf.concat([x1_2, y1_2, z1_2, x2_2, y2_2, z2_2, cube_2],axis=1)
  fc1 = tf.contrib.layers.fully_connected(conv,3000,activation_fn=tf.nn.elu)
  fc2 = tf.contrib.layers.fully_connected(fc1, 3000,activation_fn=tf.nn.elu)
  logits = tf.contrib.layers.fully_connected(fc2,135,activation_fn=tf.nn.elu)
  soft_logits = tf.nn.softmax(logits)

  #fc2 = tf.nn.dropout(fc2, keep_prob)
  loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tf.cast(Y_,tf.int32)))
  optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)
  #optimizer = tf.train.AdamOptimizer(lr).minimize(loss)


In [6]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
gpu_options = tf.GPUOptions(allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(init)

def whitening(image):
    """Whitening. Normalises image to zero mean and unit variance."""

    image = image.astype(np.float32)

    mean = np.mean(image)
    std = np.std(image)

    if std > 0:
        ret = (image - mean) / std
    else:
        ret = image * 0.
    return ret


dataset = np.load("drive/My Drive/Colab Notebooks/Dataset_final_1.npy")
npad = ((43,43), (43,43), (43,43))
#for i in range (0, len(dataset)):
#  dataset[i][0] = whitening(dataset[i][0])
print("dataset length ",len(dataset))
training_set = []

hm_epochs = 1
starting = 0

if(starting>0):
  saver.restore(sess, "drive/My Drive/Colab Notebooks/Training Models/model-"+ str(starting)+".ckpt")
  print("Model Restored")
print("Start Training")
try:
  for epoch in range(hm_epochs):
      #print("Start Training")
      correct = 0
      iter = 0
      stime = time.time()
      for i in range(0,len(dataset)):
        total = np.size(dataset[i][0])
        print("File " + str(i))
        for j in range (0,1000):
          value = 0
          stime = time.time()
          """
          voxel = random.randint(0,total)

          #find the voxel x,y,z values
          z_layer = (voxel//65536)
          row = (voxel%65536)//256
          element = (voxel%65536)%256
          value = dataset[i][1][z_layer][row][element]
          """
          while(value==0):
            voxel = random.randint(0,total)
            output = np.zeros((1,135),dtype=np.int32)

            #find the voxel x,y,z values
            z_layer = (voxel//65536)
            row = (voxel%65536)//256
            element = (voxel%65536)%256
            value = dataset[i][1][z_layer][row][element]

          
          raw = np.pad(dataset[i][0], pad_width=npad, mode='constant', constant_values=0)
          #raw = raw/5500
          raw = whitening(raw)
          n_values = 135
          output = [value]

          small_x = raw[z_layer+43][row+29:row+58,element+29:element+58]
          small_y = raw[z_layer+29:z_layer+58,row+29:row+58,element+43]
          small_z = raw[z_layer+29:z_layer+58,row+43,element+29:element+58]
          large_x = raw[z_layer][row:row+87,element:element+87]
          large_y = raw[z_layer:z_layer+87,row:row+87,element]
          large_z = raw[z_layer:z_layer+87,row,element:element+87]
          vol = raw[z_layer+37:z_layer+50,row+37:row+50,element+37:element+50]


          small_x = np.reshape(small_x, (1,29,29,1))
          small_y = np.reshape(small_y, (1,29,29,1))
          small_z = np.reshape(small_z, (1,29,29,1))
          large_x = np.reshape(large_x, (1,87,87,1))
          large_y = np.reshape(large_y, (1,87,87,1))
          large_z = np.reshape(large_z, (1,87,87,1))
          vol = np.reshape(vol, (1,13,13,13,1))
          pred = (soft_logits.eval({x1: small_x, y1: small_y, z1: small_z, x2: large_x, y2: large_y, z2: large_z, cube: vol},session=sess))
          #print(pred)
          print("Precited: ", np.argmax(pred))
          _,loss_value = sess.run([optimizer, loss], feed_dict={x1: small_x, y1: small_y, z1: small_z, x2: large_x, y2: large_y, z2: large_z, cube: vol , Y_ : output, lr: 0.001})
          print("Class: ", value)
          #output the probabilities
          #pred = (soft_logits.eval({x1: small_x, y1: small_y, z1: small_z, x2: large_x, y2: large_y, z2: large_z, cube: vol},session=sess))
          #print(pred)
          #print("Precited: ", np.argmax(pred))
          if(value==np.argmax(pred)):
            correct += 1
          iter += 1
          print("Correct Prediction % : ", (correct/iter)*100)
          if((j+1)%100 == 0):
            print(str(j+1) + " complete out of 20000. Loss is: " + str(loss_value))
          if((j+1)%5000 == 0):
            print("Training 5000 iterations complete Saving Model with " + str(j+1) +" Iterations")
            save_path = saver.save(sess, "drive/My Drive/Colab Notebooks/Training Models/model-"+ str(j+1)+".ckpt")
            print("Model saved in path: %s" % save_path)

            
          #print(stime-time.time())
            
  print("One Epoch Completed: Time Taken %s s" % (time.time()-stime))
  stime = time.time()
  print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',str(loss_value))


  print("Training Complete Saving Model with " + str(j+1) +" Iterations")
  save_path = saver.save(sess, "drive/My Drive/Colab Notebooks/Training Models/model-"+ str(j+1)+".ckpt")
  print("Model saved in path: %s" % save_path)


  print("Done!")

except KeyboardInterrupt:
  print("Training Interrupted Saving Model with " + str(j+1+starting) +" Iterations")
  #save_path = saver.save(sess, "drive/My Drive/Colab Notebooks/Training Models/model-"+ str(j+1+starting)+".ckpt")
  #print("Model saved in path: %s" % save_path)

dataset length  5
Start Training
File 0
Precited:  127
Class:  98.0
Correct Prediction % :  0.0
Precited:  98
Class:  15.0
Correct Prediction % :  0.0
Precited:  15
Class:  15.0
Correct Prediction % :  33.33333333333333
Training Interrupted Saving Model with 4 Iterations


This is the Evaluation Code

In [7]:
for i in range(1,len(dataset)):
  total = np.size(dataset[i][0])
  print("File " + str(i))
  iter = 0
  correct = 0
  output = np.zeros((65536,), dtype=int)
  for j in range (6553600,6619137):
    stime = time.time()
    voxel = j
    print(j)
    
    
    
    #find the voxel x,y,z values
    z_layer = (voxel//65536)
    row = (voxel%65536)//256
    element = (voxel%65536)%256
    raw = np.pad(dataset[i][0], pad_width=npad, mode='constant', constant_values=0)
    raw = whitening(raw)
    value = dataset[i][1][z_layer][row][element]
    if(value==0):
      iter += 1
      output[iter] = 0
      continue
    
    small_x = raw[z_layer+43][row+29:row+58,element+29:element+58]
    small_y = raw[z_layer+29:z_layer+58,row+29:row+58,element+43]
    small_z = raw[z_layer+29:z_layer+58,row+43,element+29:element+58]
    large_x = raw[z_layer][row:row+87,element:element+87]
    large_y = raw[z_layer:z_layer+87,row:row+87,element]
    large_z = raw[z_layer:z_layer+87,row,element:element+87]
    vol = raw[z_layer+37:z_layer+50,row+37:row+50,element+37:element+50]
    
    small_x = np.reshape(small_x, (1,29,29,1))
    small_y = np.reshape(small_y, (1,29,29,1))
    small_z = np.reshape(small_z, (1,29,29,1))
    large_x = np.reshape(large_x, (1,87,87,1))
    large_y = np.reshape(large_y, (1,87,87,1))
    large_z = np.reshape(large_z, (1,87,87,1))
    vol = np.reshape(vol, (1,13,13,13,1))
    

    pred = (soft_logits.eval({x1: small_x, y1: small_y, z1: small_z, x2: large_x, y2: large_y, z2: large_z, cube: vol},session=sess))
    #print(pred)
    print(str(iter+1) + " out of " + str(total))
    print("Value: ", value)
    print("Precited: ", np.argmax(pred))
    
    output[iter] = np.argmax(pred)
    
    if(value==np.argmax(pred)):
      correct += 1
    iter += 1
    print("Correct Prediction % : ", (correct/iter)*100)
    print("One Voxel Completed: Time Taken %s s" % (time.time()-stime))
    stime = time.time()

File 1
6553600
6553601
6553602
6553603
6553604
6553605
6553606
6553607
6553608
6553609
6553610


KeyboardInterrupt: ignored

In [0]:
import tensorflow as tf

true_labels = [4, 11, 23, 30, 31, 32, 35, 36, 37, 38, 39, 40, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 55, 56, 57,
                   58, 59, 60, 61, 62, 69, 71, 72, 73, 75, 76, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 112,
                   113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 128, 129, 132, 133, 134, 135, 136,
                   137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
                   157, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
                   179, 180, 181, 182, 183, 184, 185, 186, 187, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,
                   201, 202, 203, 204, 205, 206, 207]

print(len(true_labels))

onehot_labels=tf.one_hot(indices=[1],depth=134)
print(onehot_labels)

In [0]:
def eval(true,predicted):
  true_labels = [4, 11, 23, 30, 31, 32, 35, 36, 37, 38, 39, 40, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 55, 56, 57,
                   58, 59, 60, 61, 62, 69, 71, 72, 73, 75, 76, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 112,
                   113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 128, 129, 132, 133, 134, 135, 136,
                   137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
                   157, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
                   179, 180, 181, 182, 183, 184, 185, 186, 187, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,
                   201, 202, 203, 204, 205, 206, 207]
  
  dices = []
  tester = []
  
  
  for num in (true_labels):
    if (num in true):
      masker = (true==num)
      true_mask = masker*true
      pred_mask = masker*predicted
      numerator = 2*np.count_nonzero(pred_mask==num)
      
      denominator = np.count_nonzero(true==num)+np.count_nonzero(predicted==num)
      dice = numerator/denominator
      dices.append(dice)
      tester.append([num,dice,numerator,denominator])
  #print(tester)
  return sum(dices)/len(dices)

The Following is a Experimental code for running on the server

In [0]:
import tensorflow as tf
import numpy as np
import random
import time
from numpy import array
import keras
import np_utils
import sys

tf.reset_default_graph()
x1 = tf.placeholder(tf.float32)
y1 = tf.placeholder(tf.float32)
z1 = tf.placeholder(tf.float32)
x2 = tf.placeholder(tf.float32)
y2 = tf.placeholder(tf.float32)
z2 = tf.placeholder(tf.float32)
cube = tf.placeholder(tf.float32)
lr = tf.placeholder(tf.float32)
Y_ = tf.placeholder(tf.int32)
training = tf.placeholder(tf.bool, name='training')
keep_prob = tf.placeholder(tf.float32)

x1 = tf.reshape(x1, shape=[1,29,29,1])
y1 = tf.reshape(y1, shape=[1,29,29,1])
z1 = tf.reshape(z1, shape=[1,29,29,1])
x2 = tf.reshape(x2, shape=[1,87,87,1])
y2 = tf.reshape(y2, shape=[1,87,87,1])
z2 = tf.reshape(z2, shape=[1,87,87,1])
cube = tf.reshape(cube, shape=[1,13,13,13,1])

def conv3d(x, n_kernels):
  return tf.layers.conv3d(x,n_kernels,(5,5,5),padding="valid",activation=tf.nn.elu)

def conv2d(x, n_kernels):
  return tf.layers.conv2d(x,n_kernels,(5,5),padding="valid",activation=tf.nn.elu)

def conv2d_shared(x, n_kernels,nam):
  return tf.layers.conv2d(x,n_kernels,(5,5),padding="valid",name=nam)

def maxpool(x):
  return tf.layers.max_pooling2d(x,(2,2),padding="valid")

def meanpool(x):
  return tf.layers.average_pooling2d(x,(3,3),strides=3)

def batchnorm(x, training):
  return tf.layers.batch_normalization(x, training=training)
    
def relu(x):
  return tf.nn.elu(x)

def residual(x,y):
  n_x = x.shape[-1]
  n_y = y.shape[-1]
  spatial_rank = 3

  if n_x > n_y:  # pad the channel dim
      pad_1 = np.int((n_x - n_y) // 2)
      pad_2 = np.int(n_x - n_y - pad_1)
      padding_dims = np.vstack(([[0, 0]], [[0, 0]] * spatial_rank, [[pad_1, pad_2]]))
      y = tf.pad(tensor=y, paddings=padding_dims.tolist(), mode='CONSTANT')
  elif n_x < n_y:  # make a projection
      y = conv3d(y,1)

  # element-wise sum of both paths
  output_tensor = x + y
  

  return output_tensor

def dilatedCon(x,n_kernels,dilation):
  return tf.layers.conv3d(x, n_kernels,(3,3,3), [1, 1, 1], padding="same", dilation_rate=[dilation, dilation, dilation])

def finalconv(x):
  return tf.layers.conv3d(x,1,(1,1,1),padding="same")

def softmax(x):
  return tf.nn.softmax(x)


with tf.device('/gpu:0'):
  with tf.variable_scope("conv1",reuse=tf.AUTO_REUSE):
    x1_1 = conv2d_shared(x1,20,"layer1")
    y1_1 = conv2d_shared(y1,20,"layer1")
    z1_1 = conv2d_shared(z1,20,"layer1")
  with tf.variable_scope("conv2",reuse=tf.AUTO_REUSE):
    x2_1 = conv2d_shared(meanpool(x2),20,"layer2")
    y2_1 = conv2d_shared(meanpool(y2),20,"layer2")
    z2_1 = conv2d_shared(meanpool(z2),20,"layer2")

  cube_1 = conv3d(cube,20)
  """
  x1_1 = batchnorm(x1_1,training)
  y1_1 = batchnorm(y1_1,training)
  z1_1 = batchnorm(z1_1,training)
  x2_1 = batchnorm(x2_1,training)
  y2_1 = batchnorm(y2_1,training)
  z2_1 = batchnorm(z2_1,training)
  cube_1 = batchnorm(cube_1,training)
  """
  x1_2 = conv2d(x1_1,50)
  y1_2 = conv2d(y1_1,50)
  z1_2 = conv2d(z1_1,50)
  x2_2 = conv2d(x2_1,50)
  y2_2 = conv2d(y2_1,50)
  z2_2 = conv2d(z2_1,50)
  cube_2 = conv3d(cube_1,50)
  
  x1_2 = tf.layers.flatten(x1_2)
  y1_2 = tf.layers.flatten(y1_2)
  z1_2 = tf.layers.flatten(z1_2)
  x2_2 = tf.layers.flatten(x2_2)
  y2_2 = tf.layers.flatten(y2_2)
  z2_2 = tf.layers.flatten(z2_2)
  cube_2 = tf.layers.flatten(cube_2)

  conv = tf.concat([x1_2, y1_2, z1_2, x2_2, y2_2, z2_2, cube_2],axis=1)
  """
  conv = tf.layers.batch_normalization(conv, training=training)
  """
  fc1 = tf.contrib.layers.fully_connected(conv,3000,activation_fn=None)
  fc1 = tf.layers.batch_normalization(fc1, training=training)
  fc1 = relu(fc1)
  
  fc2 = tf.contrib.layers.fully_connected(fc1, 3000,activation_fn=None)
  fc2 = tf.layers.batch_normalization(fc2, training=training)
  fc2 = relu(fc2)
  
  logits = tf.contrib.layers.fully_connected(fc2,135,activation_fn=tf.nn.elu)
  soft_logits = tf.nn.softmax(logits)

  loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tf.cast(Y_,tf.int32)))
  optimizer = tf.train.GradientDescentOptimizer(lr)
  optimizer = optimizer.minimize(loss)

saver = tf.train.Saver()
init = tf.global_variables_initializer()
gpu_options = tf.GPUOptions(allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(init)

def whitening(image):
    """Whitening. Normalises image to zero mean and unit variance."""

    image = image.astype(np.float32)

    mean = np.mean(image)
    std = np.std(image)

    if std > 0:
        ret = (image - mean) / std
    else:
        ret = image * 0.
    return ret


dataset = np.load("drive/My Drive/Colab Notebooks/Dataset_final_1.npy")
npad = ((43,43), (43,43), (43,43))
print("dataset length ",len(dataset))
training_set = []

hm_epochs = 1
starting = 0
no_samples = 10000

if(starting>0):
  saver.restore(sess, "Training Models/model-"+ str(starting)+".ckpt")
  print("Model Restored")

print("Start Training")
try:
  for epoch in range(hm_epochs):
      e_time = time.time()
      #print("Start Training")
      correct = 0
      iter = 0
      stime = time.time()
      for i in range(0,len(dataset)):
        total = np.size(dataset[i][0])
        print("File " + str(i))
        
        for j in range (0,1000):
          value = 0
          stime = time.time()
          while(value==0):
            voxel = random.randint(0,total)
            output = np.zeros((1,135),dtype=np.int32)

            #find the voxel x,y,z values
            z_layer = (voxel//65536)
            row = (voxel%65536)//256
            element = (voxel%65536)%256
            value = dataset[i][1][z_layer][row][element]

          
          raw = np.pad(dataset[i][0], pad_width=npad, mode='constant', constant_values=0)
          #raw = raw/5500
          raw = whitening(raw)
          n_values = 135
          output = [value]

          small_x = raw[z_layer+43][row+29:row+58,element+29:element+58]
          small_y = raw[z_layer+29:z_layer+58,row+29:row+58,element+43]
          small_z = raw[z_layer+29:z_layer+58,row+43,element+29:element+58]
          large_x = raw[z_layer][row:row+87,element:element+87]
          large_y = raw[z_layer:z_layer+87,row:row+87,element]
          large_z = raw[z_layer:z_layer+87,row,element:element+87]
          vol = raw[z_layer+37:z_layer+50,row+37:row+50,element+37:element+50]


          small_x = np.reshape(small_x, (1,29,29,1))
          small_y = np.reshape(small_y, (1,29,29,1))
          small_z = np.reshape(small_z, (1,29,29,1))
          large_x = np.reshape(large_x, (1,87,87,1))
          large_y = np.reshape(large_y, (1,87,87,1))
          large_z = np.reshape(large_z, (1,87,87,1))
          vol = np.reshape(vol, (1,13,13,13,1))
          
          
          pred = (soft_logits.eval({x1: small_x, y1: small_y, z1: small_z, x2: large_x, y2: large_y, z2: large_z, cube: vol, keep_prob: 1.0, training:False },session=sess))
          #print(pred)
          print("Precited: ", np.argmax(pred))
          _,loss_value = sess.run([optimizer, loss], feed_dict={x1: small_x, y1: small_y, z1: small_z, x2: large_x, y2: large_y, z2: large_z, cube: vol , Y_ : output, lr:0.01 , keep_prob:0.9, training:True})
          print("Class: ", value)
          print(time.time()-stime)
          print(str(loss_value))
          if(value==np.argmax(pred)):
            correct += 1
          iter += 1
          print("Correct Prediction % : ", (correct/iter)*100)
          if((j+1)%100 == 0):
            print(str(j+1) + " complete out of 20000. Loss is: " + str(loss_value))
          if((j+1)%5000 == 0):
            print("Training 5000 iterations complete Saving Model with " + str(j+1) +" Iterations")
            save_path = saver.save(sess, "Training Models/model-"+ str(j+1)+".ckpt")
            print("Model saved in path: %s" % save_path)

            
          
            
      print("One Epoch Completed: Time Taken %s s" % (time.time()-e_time))
      e_time = time.time()
      print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',str(loss_value))


  print("Training Complete Saving Model with " + str((iter+1)*hm_epochs*len(dataset)) +" Iterations")
  save_path = saver.save(sess, "Training Models/model-"+ str((iter+1)*hm_epochs*len(dataset))+".ckpt")
  print("Model saved in path: %s" % save_path)


  print("Done!")

except KeyboardInterrupt:
  print("Training Interrupted Saving Model with " + str(j+1+starting) +" Iterations")
  #save_path = saver.save(sess, "Training Models/model-"+ str(j+1+starting)+".ckpt")
  #print("Model saved in path: %s" % save_path)

In [0]:
import tensorflow as tf
with tf.Session() as sess:
  devices = sess.list_devices()
  
print(devices)

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 7531884453388044863), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8779326169682415652), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 634049089981660065), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 11276946637, 16859271132039702870)]
